<a href="https://colab.research.google.com/github/julie-b05302301/bda/blob/main/Copy_of_BDA_Final0602_nopart3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part1:商品向量

Link to google drive

In [1]:
# 環境設定
import os
import pandas as pd
import sys
import numpy as np
import re
import pickle

In [ ]:
# !pip install -U --no-cache-dir gdown --pre
# !pip install --upgrade --no-cache-dir gdown
from google.colab import drive
drive.mount('/content/gdrive')
!ls


import os
import pandas as pd
raw_path = os.path.join(os.path.join('/content/gdrive/MyDrive/BDA_Final/Raw'))
test_file = pd.read_csv(os.path.join(raw_path, '91APP_DataSet_2022/91APP_SalePageData.csv'))
print(test_file)

MessageError: ignored

In [ ]:
!gdown --id 1YVz3rpIKtZVxC7pxEmTUnpRPtdnGHoEn --output SalePageData.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1YVz3rpIKtZVxC7pxEmTUnpRPtdnGHoEn
To: /content/SalePageData.csv
100% 1.55M/1.55M [00:00<00:00, 144MB/s]


Part 1: 斷詞->產生各詞向量->產生各商品矩陣

In [ ]:
# Read SalePage: SalePageId <-> SalePageTitle
import os
import sys
import pandas as pd
import numpy as np

pages = pd.read_csv(os.path.join('/content', 'SalePageData.csv'))

pages # (17742, 2)

,SalePageId,SalePageTitle
0,Q1hQKLfmnu1n1twlQk7jvA==,123
1,Y7nL1VjgNwvsHYTfVeuHuw==,Ann'S品牌止滑墊
2,708J4yBDWIZQiTcisLdcJw==,Care bear海灘提袋
3,CD1r8DeKFSNRTI0fzFcvEw==,ANNS兔子耳朵靴撐🐰
4,6o0CygkG0GHlR9lpZ6+1yA==,Care Bears拖鞋or托特包
...,...,...
17737,JyuH8E05PbVgsJyoafSQGQ==,【直播搶先買】ANNSTAR丹妮婊姐聯名-彈力皮革貼腿防水台方頭粗跟短靴7.5cm-米白(版...
17738,/Lyj/inE8vfLius/zZtT+g==,【直播搶先買】ANNSTAR丹妮婊姐聯名-異材質拼接貼腿飛織粗跟短靴6.5cm-杏灰(版型偏小)
17739,s6mDF15Y/++PHPSIdatlVA==,【直播搶先買】ANNSTAR丹妮婊姐聯名-彈力皮革貼腿防水台方頭粗跟短靴7.5cm-咖(版型偏大)
17740,wtIcFMl76TVtiYjeTw9QgA==,【直播搶先買】ANNSTAR丹妮婊姐聯名-免綁帶帥妹平底短靴3cm-深咖


In [ ]:
# Filter out "直播" items (6550)
# Items (11192)
exclude_mask = pages['SalePageTitle'].str.contains(r'【.*】.*直播$', regex=True)
items = pages[~exclude_mask].copy()

items

,SalePageId,SalePageTitle
0,Q1hQKLfmnu1n1twlQk7jvA==,123
1,Y7nL1VjgNwvsHYTfVeuHuw==,Ann'S品牌止滑墊
2,708J4yBDWIZQiTcisLdcJw==,Care bear海灘提袋
3,CD1r8DeKFSNRTI0fzFcvEw==,ANNS兔子耳朵靴撐🐰
4,6o0CygkG0GHlR9lpZ6+1yA==,Care Bears拖鞋or托特包
...,...,...
17737,JyuH8E05PbVgsJyoafSQGQ==,【直播搶先買】ANNSTAR丹妮婊姐聯名-彈力皮革貼腿防水台方頭粗跟短靴7.5cm-米白(版...
17738,/Lyj/inE8vfLius/zZtT+g==,【直播搶先買】ANNSTAR丹妮婊姐聯名-異材質拼接貼腿飛織粗跟短靴6.5cm-杏灰(版型偏小)
17739,s6mDF15Y/++PHPSIdatlVA==,【直播搶先買】ANNSTAR丹妮婊姐聯名-彈力皮革貼腿防水台方頭粗跟短靴7.5cm-咖(版型偏大)
17740,wtIcFMl76TVtiYjeTw9QgA==,【直播搶先買】ANNSTAR丹妮婊姐聯名-免綁帶帥妹平底短靴3cm-深咖


In [ ]:
# Prerequistie for ckip-tagger
!pip install ckiptagger
!pip install tensorflow
!pip install gdown
from ckiptagger import WS, POS, NER
from ckiptagger import data_utils
data_utils.download_data_gdown("./")
ws = WS("./data")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
100%|██████████| 1.88G/1.88G [00:27<00:00, 68.2MB/s]
/usr/local/lib/python3.7/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)


In [ ]:
# Process each title
# Extract color information: -顏色 in the end of the title
# Extract size information: (版型偏大/小) after color information
# Eliminate emojis
# Tokenize with ckip-tagger
from tqdm import tqdm
import re

def process_sentence(x):
    # Extract size
    size_pattern = r'\([\u4e00-\u9fa5]*\)$'
    size_info= re.search(size_pattern, x)
    if size_info is not None: size_info = size_info.group(0)
    x = re.sub(size_pattern, '', x)

    # Clear emojis
    x = re.sub(r'[^\u4e00-\u9fa5 | \w | \' | \:| % |\.]+', ' ', x)

    # Extract color
    color_pattern = r'\s([\u4e00-\u9fa5]){1,3}$'
    color_info = re.search(color_pattern, x)
    if color_info is not None: color_info = color_info.group(0)
    x = re.sub(color_pattern, '', x)

    # Eliminate Ann's
    x = re.sub(r'^.*Ann.?s', '', x, flags=re.IGNORECASE)

    # Remove space in front/end
    x = x.strip()
    return x, size_info, color_info

def ckip_split(sentence_lst):
    ws_results = ws(sentence_lst)
    return ws_results
    
def has_word(str):
    return bool(re.search(r'[A-Z | a-z | 0-9]', str))

processed_lst = []
size_lst = []
color_lst = []
for title in tqdm(items['SalePageTitle']):
    x, sz, cr= process_sentence(title)
    processed_lst.append(x)
    size_lst.append(sz)
    color_lst.append(cr)
items['ProcessedTitle'] = processed_lst
items['SizeInfo'] = size_lst
items['ColorInfo'] = color_lst

split_phrases = []
long_enough = []
for idx, row in tqdm(items.iterrows(), total=items.shape[0]):
    sentence_lst = []
    if isinstance(row['ProcessedTitle'], str): sentence_lst.append(row['ProcessedTitle'])
    if isinstance(row['SizeInfo'], str): sentence_lst.append(row['SizeInfo'])
    splitted = ckip_split(sentence_lst)

    if row['SizeInfo'] is not None: splitted = splitted[0] + splitted[1]
    else: splitted = splitted[0]
    split_phrases.append(splitted)

    slices_long_enough = [i.strip(' -') for i in splitted if len(i.strip(' -')) >= 2 and not has_word(i.strip())]
    if row['ColorInfo'] is not None: slices_long_enough += [row['ColorInfo'].strip()]
    long_enough.append(slices_long_enough)
    pass
items['SplitPhrases'] = split_phrases
items['LongEnough'] = long_enough

items

 61%|██████    | 6850/11192 [05:43<07:12, 10.04it/s]

In [ ]:
phrases_series = items['LongEnough']
phr_set = [] # 3630 # pseudo-set, actually a list

for phrases_lst in tqdm(items['LongEnough']):
    for phr in phrases_lst:
        if phr not in phr_set: phr_set.append(phr)
# phrases = pd.Series(list(phr_set)).to_frame('Phrase')

print(phr_set)

100%|██████████| 11192/11192 [00:01<00:00, 7390.77it/s]

['品牌', '止滑墊', '海灘', '提袋', '兔子', '耳朵靴', '拖鞋', '托特', '專屬', '梗紅包袋', '版面', '紅色', '購物袋', '鞋蜜粉', '隨身', '口味', '隨機', '三部曲', '面膜', '白鞋', '救星組', '污潔', '白慕絲', '心機', '珠絲', '直播', '限定', '字母', '項鍊', '贈品卷', '商品', '防水', '噴霧', '收納', '充氣', '頸枕', '贈品', '貓咪', '手繪', '印花', '郊遊', '背包', '單件', '泰迪', '毛水桶', '包贈', '品卷', '帆布', '方形', '餐袋', '長效性', '市售', '小白鞋', '家族', '防髒', '防汙', '萬聖', '刺青', '貼紙', '舒適', '乳膠', '鞋子', '大半號', '專用', '彈力', '机密', '膠原', '美妍', '凍乙盒', '市價', '流蘇', '出貨', '挑色', '加碼', '訂製', '甜美', '限量', '斯亞', '親筆', '簽名', '手寫', '明信片', '寶貝', '鞋組', '極效', '靴款', '尺寸', '小美人', '皮革', '護理組', '即可', '東京', '來回', '機票', '開獎', '確幸', '訂單', '髮圈組', '彭彭', '聯名', '長靴', '禮盒組', '附贈', '質感', '伸縮', '捲尺', '聯名鞋', '時髦', '漸層', '金邊', '墨鏡組', '萬聖趴', '造型', '發光', '髮箍', '贈送', '替換', '鞋帶', '口袋', '聯名靴', '熊毛茸茸', '軟呼', '漁夫帽', '人生', '仙玫', '金仙女', '涼鞋', '服務', '懶人鞋', '粉嫩', '毛球', '單筆', '累送', '紅包袋', '彩虹', '雲朵', '環保', '餐具', '挑款', '最愛', '足弓', '紓壓', '鞋墊', '備註', '尺碼', '振興金', '統一', '入帳', '重複', '毛絨絨', '旅行包', '奶茶棕', '花豹', '冠軍', '心情', '變好', '笑臉襪', '俏皮', '經典', '小公主', '鑽石

In [ ]:
# Download BERT pretrain model
!wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
!unzip chinese_L-12_H-768_A-12.zip

--2022-05-25 01:17:41--  https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 173.194.210.128, 173.194.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381892918 (364M) [application/zip]
Saving to: ‘chinese_L-12_H-768_A-12.zip’

chinese_L-12_H-768_ 100%[===================>] 364.20M   117MB/s    in 3.1s    

2022-05-25 01:17:44 (117 MB/s) - ‘chinese_L-12_H-768_A-12.zip’ saved [381892918/381892918]

Archive:  chinese_L-12_H-768_A-12.zip
   creating: chinese_L-12_H-768_A-12/
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-12/bert_config.json  


In [ ]:
# BERT: generate word vector
!pip install keras-bert
import tensorflow
from keras_bert import load_vocabulary
from keras_bert import Tokenizer
from keras_bert import load_trained_model_from_checkpoint

text = phr_set

pretrained='/content/chinese_L-12_H-768_A-12'
dictpath=os.path.join(pretrained, 'vocab.txt')
config_path=os.path.join(pretrained,'bert_config.json')
checkpoint_path=os.path.join(pretrained,'bert_model.ckpt')
print("The pretrained model is loaded")
# 載入中文預訓練好的BERT語料庫

model = load_trained_model_from_checkpoint(config_path, checkpoint_path)
bert_token_dict=load_vocabulary(dictpath)
bert_tokenizer = Tokenizer(bert_token_dict)

# Estimated 6 hours
num = len(text)
data=np.zeros((num ,768))  # 建立空的nparray
for i in tqdm(range(0, num)):
	token = bert_tokenizer.tokenize(text[i])
	indices, segments=bert_tokenizer.encode(first=text[i], max_len=512)
	# 轉化成向量(向量維度768是預設輸出長度)
	predicts = model.predict([np.array([indices]), np.array([segments])])[0]
	embedding=predicts[0]
	data[i,:]=embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The pretrained model is loaded


100%|██████████| 3630/3630 [2:09:52<00:00,  2.15s/it]


In [ ]:
# Save output matrix
import datetime
import pytz
from google.colab import files
import pickle
import numpy as np

timenow_str =  datetime.datetime.now(pytz.timezone('Asia/Taipei')).strftime("%m%d%H%M")

np.save('array{}.npy'.format(timenow_str), data)
files.download('array{}.npy'.format(timenow_str))

with open('phrases{}.pkl'.format(timenow_str), 'wb') as f:
  pickle.dump(text, f)
files.download('phrases{}.pkl'.format(timenow_str))

items.to_pickle('items{}.pkl'.format(timenow_str))
files.download('items{}.pkl'.format(timenow_str))


with open(os.path.join('/content/gdrive/MyDrive/BDA_Final/Results/ckpt_bert', 'array{}.npy'.format(timenow_str)), 'wb') as f:
  np.save(f, data)
with open(os.path.join('/content/gdrive/MyDrive/BDA_Final/Results/ckpt_bert', 'phrases{}.pkl'.format(timenow_str)), 'wb') as f:
  pickle.dump(text, f)
with open(os.path.join('/content/gdrive/MyDrive/BDA_Final/Results/ckpt_bert', 'items{}.pkl'.format(timenow_str)), 'wb') as f:
  items.to_pickle(f)

NameError: ignored

Vector Dictionary column

In [2]:
# Load checkpoint for pervious sections
import os
!gdown --folder --id 1A9iBOw0vb4lX-QvI3sOsoBrVHr27KNmz
ckpt_bert_pth = '/content/ckpt_bert'
version='05251237'
with open(os.path.join(ckpt_bert_pth, 'phrases{}.pkl'.format(version)), 'rb') as fp:
    phrase_list = pickle.load(fp)

with open(os.path.join(ckpt_bert_pth, 'items{}.pkl'.format(version)), 'rb') as fpl:
    item_df = pickle.load(fpl)
phrase_array = np.load(os.path.join(ckpt_bert_pth, 'array{}.npy'.format(version)))

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Retrieving folder list
Processing file 1-FpwIIi0B0t-24ohkp1pQWI87-AHhIqY array05251237.npy
Processing file 1-Ix2x1-Kdam4d3oWNU6u0dpdCUo3S2Sa items05251237.pkl
Processing file 1-JtAyt8u4ejFyxIWqemoVy3GM-3Mb1Zz phrases05251237.pkl
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-FpwIIi0B0t-24ohkp1pQWI87-AHhIqY
To: /content/ckpt_bert/array05251237.npy
100% 22.3M/22.3M [00:00<00:00, 47.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-Ix2x1-Kdam4d3oWNU6u0dpdCUo3S2Sa
To: /content/ckpt_bert/items05251237.pkl
100% 3.15M/3.15M [00:00<00:00, 54.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-JtAyt8u4ejFyxIWqemoVy3GM-3Mb1Zz
To: /content/ckpt_bert/p

In [3]:
# Generate Columns for Phrases&Vectors for each Title
phrase_dict={} 
# 字典：{標題0:{關鍵字0：向量0,關鍵字1:向量1}}

item_df.reset_index(drop=True, inplace=True)
for i in range(0,len(phrase_list)):
	phrase_dict[phrase_list[i]] = phrase_array[i]

tokens_list=item_df['LongEnough'].tolist()
title_list=item_df['SalePageTitle'].tolist()
item_vec_dict={}

for i in range(0,len(tokens_list)):#len(tokens_list)
	title=title_list[i]
	item_vec_dict[title]={}
	for p in range(0,len(tokens_list[i])):
		word=str(tokens_list[i][p])
		item_vec_dict[title][word]=phrase_dict[word]

# 在item_df新增dict欄位; dict={關鍵字0：向量0,關鍵字1:向量1}
item_df["dict"] = ""
count=0
for i in title_list:
	item_df["dict"][count]= item_vec_dict[i]
	count+=1

In [4]:
def avg_dict_values(d):
  sum_v = 0
  for k,v in d.items():
      sum_v += v
  if len(d.keys()) !=0:
    return sum_v / len(d.keys())
  else:
    return None

item_df.drop(columns=['ProcessedTitle','SizeInfo','ColorInfo','SplitPhrases'],inplace=True)
item_df.rename(columns={'LongEnough':'Splitted','dict':'Vector'},inplace=True)
# 留下SalePageId, SalePageTitle, Splitted, Vector四個欄位

item_df["Vector_avg"] = item_df.Vector.apply(lambda x:avg_dict_values(x))

In [5]:
item_df

,SalePageId,SalePageTitle,Splitted,Vector,Vector_avg
0,Q1hQKLfmnu1n1twlQk7jvA==,123,[],{},None
1,Y7nL1VjgNwvsHYTfVeuHuw==,Ann'S品牌止滑墊,"[品牌, 止滑墊]","{'品牌': [-0.7566149830818176, 0.275978147983551...","[-0.464335098862648, 0.3870873898267746, -0.14..."
2,708J4yBDWIZQiTcisLdcJw==,Care bear海灘提袋,"[海灘, 提袋]","{'海灘': [-0.41770511865615845, 0.92366778850555...","[-0.5325517952442169, 0.5189004838466644, -0.4..."
3,CD1r8DeKFSNRTI0fzFcvEw==,ANNS兔子耳朵靴撐🐰,"[兔子, 耳朵靴]","{'兔子': [-0.20414215326309204, 0.14843875169754...","[-0.3957182466983795, 0.31917642056941986, -0...."
4,6o0CygkG0GHlR9lpZ6+1yA==,Care Bears拖鞋or托特包,"[拖鞋, 托特]","{'拖鞋': [-0.3351202607154846, 0.681582272052764...","[-0.5516344010829926, 0.19937193393707275, -0...."
...,...,...,...,...,...
11187,JyuH8E05PbVgsJyoafSQGQ==,【直播搶先買】ANNSTAR丹妮婊姐聯名-彈力皮革貼腿防水台方頭粗跟短靴7.5cm-米白(版...,"[丹妮, 婊姐, 聯名, 彈力, 皮革, 防水台, 方頭, 短靴, 版型, 米白]","{'丹妮': [-0.5023792386054993, -0.25763824582099...","[-0.5033289432525635, 0.3919828023761511, -0.5..."
11188,/Lyj/inE8vfLius/zZtT+g==,【直播搶先買】ANNSTAR丹妮婊姐聯名-異材質拼接貼腿飛織粗跟短靴6.5cm-杏灰(版型偏小),"[丹妮, 婊姐, 聯名, 材質, 拼接, 貼腿, 飛織, 短靴, 版型, 杏灰]","{'丹妮': [-0.5023792386054993, -0.25763824582099...","[-0.7193372115492821, 0.49727702848613264, -0...."
11189,s6mDF15Y/++PHPSIdatlVA==,【直播搶先買】ANNSTAR丹妮婊姐聯名-彈力皮革貼腿防水台方頭粗跟短靴7.5cm-咖(版型偏大),"[丹妮, 婊姐, 聯名, 彈力, 皮革, 防水台, 方頭, 短靴, 版型, 咖]","{'丹妮': [-0.5023792386054993, -0.25763824582099...","[-0.5578294813632965, 0.4368047144263983, -0.5..."
11190,wtIcFMl76TVtiYjeTw9QgA==,【直播搶先買】ANNSTAR丹妮婊姐聯名-免綁帶帥妹平底短靴3cm-深咖,"[丹妮, 婊姐, 聯名, 綁帶, 帥妹, 平底, 短靴, 深咖]","{'丹妮': [-0.5023792386054993, -0.25763824582099...","[-0.581490583717823, 0.4585170275531709, -0.57..."


In [6]:
# Generate a Dictionary of Phrases(3630) to Vector(768,)
phrase_vec_dict = {}

for i in range(len(phrase_list)):
  phrase_vec_dict[phrase_list[i]] = phrase_array[i, :]

phrase_vec_df = pd.DataFrame(phrase_vec_dict.items(), columns=['Phrase', 'Vector'])

In [7]:
# Save Output Dataframe: ID, Title, Splitted(list of phrases), Vector(dictionary with Phrases(appear in title) <-> Vector (BERT generated (768,)))
import datetime
import pytz
from google.colab import files
import pickle

timenow_str =  datetime.datetime.now(pytz.timezone('Asia/Taipei')).strftime("%m%d%H%M")

item_df.to_pickle('SalePageWithPhrasesVectors{}.pkl'.format(timenow_str))
files.download('SalePageWithPhrasesVectors{}.pkl'.format(timenow_str))

phrase_vec_df.to_pickle('PhraseVector{}.pkl'.format(timenow_str))
files.download('PhraseVector{}.pkl'.format(timenow_str))

item_df.to_pickle(os.path.join('/content/gdrive/MyDrive/BDA_Final/Results/Items&Phrases', 'SalePageWithPhrasesVectors{}.pkl'.format(timenow_str)))
phrase_vec_df.to_pickle(os.path.join('/content/gdrive/MyDrive/BDA_Final/Results/Items&Phrases', 'PhraseVector{}.pkl').format(timenow_str))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FileNotFoundError: ignored

# Part2: 會員向量

In [8]:
# 環境設定
import os
import pandas as pd
import sys
import numpy as np
import re
import pickle

In [9]:
# 載入Part1 的 output：OneVectorVersion dataframe,TwoVectorVersion dataframe
!gdown --folder --id 1VSrGhg_WLbpep0D9i4thlaz8eJLGm8wL
part1_path = '/content/Items&Phrases'
version = '05251401'
item_df = pd.read_pickle(os.path.join(part1_path, 'SalePageWithPhrasesVectors{}.pkl'.format(version)))
phrase2vector_df= pd.read_pickle(os.path.join(part1_path,  'PhraseVector{}.pkl'.format(version)))

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Retrieving folder list
Processing file 1Y9mrSjNbrKeW_6Fqm8YYUhLGnod3rOBb itemTitle_df.pkl
Processing file 1-4mGTpl0rD0iHV2pt96KhPoUKm-gFhgH PhraseVector05251401.pkl
Processing file 1--CTUgopai8-7e9SaxUDF2N_ODatvXt5 SalePageWithPhrasesVectors05251401.pkl
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1Y9mrSjNbrKeW_6Fqm8YYUhLGnod3rOBb
To: /content/Items&Phrases/itemTitle_df.pkl
100% 72.3M/72.3M [00:02<00:00, 29.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-4mGTpl0rD0iHV2pt96KhPoUKm-gFhgH
To: /content/Items&Phrases/PhraseVector05251401.pkl
100% 22.5M/22.5M [00:00<00:00, 54.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--CTUgopai8-7e9Sax

In [ ]:
''' 
Desciption: Aggregated Version of the original SalePageData, information including: 
  SalePageId:[String] Can be mapped to 'SalePageId' in behavior data <-話說好像有些viewproduct、add、checkout、purchase的行為有SalePAgeId, 但是對應不到SalePageTitle
  SalePageTitle: [String]
  Splitted: [list] 斷詞並篩選後的剩下的詞
  Vector: [Dict] 每個Splitted裡的詞對應的向量(768, )
'''
item_df

,SalePageId,SalePageTitle,Splitted,Vector
0,Q1hQKLfmnu1n1twlQk7jvA==,123,[],{}
1,Y7nL1VjgNwvsHYTfVeuHuw==,Ann'S品牌止滑墊,"[品牌, 止滑墊]","{'品牌': [-0.7566149830818176, 0.275978147983551..."
2,708J4yBDWIZQiTcisLdcJw==,Care bear海灘提袋,"[海灘, 提袋]","{'海灘': [-0.41770511865615845, 0.92366778850555..."
3,CD1r8DeKFSNRTI0fzFcvEw==,ANNS兔子耳朵靴撐🐰,"[兔子, 耳朵靴]","{'兔子': [-0.20414215326309204, 0.14843875169754..."
4,6o0CygkG0GHlR9lpZ6+1yA==,Care Bears拖鞋or托特包,"[拖鞋, 托特]","{'拖鞋': [-0.3351202607154846, 0.681582272052764..."
...,...,...,...,...
11187,JyuH8E05PbVgsJyoafSQGQ==,【直播搶先買】ANNSTAR丹妮婊姐聯名-彈力皮革貼腿防水台方頭粗跟短靴7.5cm-米白(版...,"[丹妮, 婊姐, 聯名, 彈力, 皮革, 防水台, 方頭, 短靴, 版型, 米白]","{'丹妮': [-0.5023792386054993, -0.25763824582099..."
11188,/Lyj/inE8vfLius/zZtT+g==,【直播搶先買】ANNSTAR丹妮婊姐聯名-異材質拼接貼腿飛織粗跟短靴6.5cm-杏灰(版型偏小),"[丹妮, 婊姐, 聯名, 材質, 拼接, 貼腿, 飛織, 短靴, 版型, 杏灰]","{'丹妮': [-0.5023792386054993, -0.25763824582099..."
11189,s6mDF15Y/++PHPSIdatlVA==,【直播搶先買】ANNSTAR丹妮婊姐聯名-彈力皮革貼腿防水台方頭粗跟短靴7.5cm-咖(版型偏大),"[丹妮, 婊姐, 聯名, 彈力, 皮革, 防水台, 方頭, 短靴, 版型, 咖]","{'丹妮': [-0.5023792386054993, -0.25763824582099..."
11190,wtIcFMl76TVtiYjeTw9QgA==,【直播搶先買】ANNSTAR丹妮婊姐聯名-免綁帶帥妹平底短靴3cm-深咖,"[丹妮, 婊姐, 聯名, 綁帶, 帥妹, 平底, 短靴, 深咖]","{'丹妮': [-0.5023792386054993, -0.25763824582099..."


In [ ]:
'''
Discription: 詞(不重複，共3630個) 對應到的 Vector(BERT生成，每個768維)
或許你們用的到?
'''
phrase2vector_df

,Phrase,Vector
0,品牌,"[-0.7566149830818176, 0.275978147983551, 0.258..."
1,止滑墊,"[-0.1720552146434784, 0.49819663166999817, -0...."
2,海灘,"[-0.41770511865615845, 0.9236677885055542, -0...."
3,提袋,"[-0.6473984718322754, 0.11413317918777466, -0...."
4,兔子,"[-0.20414215326309204, 0.14843875169754028, -0..."
...,...,...
3625,英文字,"[-1.1670032739639282, 0.2552802860736847, -0.4..."
3626,巴洛克蕾絲,"[-0.6859194040298462, -0.13565593957901, 0.049..."
3627,珍珠紗花苞,"[-0.42280036211013794, -0.41334328055381775, -..."
3628,婊姐,"[-0.6613941788673401, 0.03393767401576042, -0...."


#### 讀入會員行為資料(201806-202012)

In [ ]:
# 201806-201812
!gdown --id '1MbGXfWAwTe7GdAxmyyS9i3JfC9z_9l7w' --output 91APP_BehaviorData_20180601.csv
!gdown --id '1B_LNLxeEQFmJq6_GqJMsurASUbntXpGn' --output 91APP_BehaviorData_20180701.csv
!gdown --id '1YOWE7clQ6R7F0C7zxolx8_H7t9IbimZl' --output 91APP_BehaviorData_20180801.csv
!gdown --id '1OOosr9kcJttOPQr55Si5ImCHi-HVCtTj' --output 91APP_BehaviorData_20180901.csv
!gdown --id '14EnXGrQ2rItnMw6FVylkAuuTNgmjvYaX' --output 91APP_BehaviorData_20181001.csv
!gdown --id '13nE7MxOr5gpOPGPO9746jJNF_YkRbcsv' --output 91APP_BehaviorData_20181101.csv
!gdown --id '1aeQZDBJghtkXH1Ul34GCdjvjMMcsNkD0' --output 91APP_BehaviorData_20181201.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1MbGXfWAwTe7GdAxmyyS9i3JfC9z_9l7w
To: /content/91APP_BehaviorData_20180601.csv
100% 48.4M/48.4M [00:00<00:00, 193MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1B_LNLxeEQFmJq6_GqJMsurASUbntXpGn
To: /content/91APP_BehaviorData_20180701.csv
100% 296M/296M [00:01<00:00, 176MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=Fut

In [ ]:
# 201901-201912
!gdown --id '1nuPR_g4UKR8JjBbtXZtIpLpnlg-LmwFf' --output 91APP_BehaviorData_20190101.csv
!gdown --id '1WOkXxrgAtwT5X34rneoklLTJ5Ux21v3z' --output 91APP_BehaviorData_20190201.csv
!gdown --id '1JHR3_hZi5A58H6J2-c-AQZx3DD-TlCcw' --output 91APP_BehaviorData_20190301.csv
!gdown --id '13-2X4xs1H9qT7qjSJecOlJvhjV7_hWjA' --output 91APP_BehaviorData_20190401.csv
!gdown --id '18pX-Pd6v9d_dQ1h2wNFYk6L5HQ9RPLv_' --output 91APP_BehaviorData_20190501.csv
!gdown --id '1IaC3fM7DsGo91EaiFWHBFRKj4eimiSjR' --output 91APP_BehaviorData_20190601.csv
!gdown --id '1KdC-V2FE9KSXmpQmdppZ30zSSeyqMgrk' --output 91APP_BehaviorData_20190701.csv
!gdown --id '1ZKL2hJopqRTrZxBCFvqCrP9Xqi_neAt4' --output 91APP_BehaviorData_20190801.csv
!gdown --id '1SK4cMqMMfN7gamLyFUC4pMLJc0WirmRt' --output 91APP_BehaviorData_20190901.csv
!gdown --id '17kH5eF9JmAAlzAKUqZLe4q6d8Yz0qB7e' --output 91APP_BehaviorData_20191001.csv
!gdown --id '1niVVjV_n15zSq8sln9i4fc-GFNs2dOMP' --output 91APP_BehaviorData_20191101.csv
!gdown --id '1e5DO0S7aMz2uFVFw4Sjn73J6QXzP89gO' --output 91APP_BehaviorData_20191201.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1nuPR_g4UKR8JjBbtXZtIpLpnlg-LmwFf
To: /content/91APP_BehaviorData_20190101.csv
100% 396M/396M [00:03<00:00, 101MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1WOkXxrgAtwT5X34rneoklLTJ5Ux21v3z
To: /content/91APP_BehaviorData_20190201.csv
100% 318M/318M [00:02<00:00, 115MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=Futur

In [ ]:
# 202001-202006
!gdown --id '1F_1gAyEtmlW6NRQaHySCRLWtyFmN_7IK' --output 91APP_BehaviorData_20200101.csv
!gdown --id '1RZavefjlWwECDTHGtzT751HNg4h8m_BO' --output 91APP_BehaviorData_20200201.csv
!gdown --id '1e4C7YARl6ZICO3NYr740NGhx7QQn3jJs' --output 91APP_BehaviorData_20200301.csv
!gdown --id '1VNJSqsJpV-CtRse7aA5K0GPDMWloxEml' --output 91APP_BehaviorData_20200401.csv
!gdown --id '1DAbArrKDDmAefsZBDZk7SXeg0eYKNgWa' --output 91APP_BehaviorData_20200501.csv
!gdown --id '1_gxCmW5vRSozvrE23HdAX7yZS6mhaDXt' --output 91APP_BehaviorData_20200601.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1F_1gAyEtmlW6NRQaHySCRLWtyFmN_7IK
To: /content/91APP_BehaviorData_20200101.csv
100% 447M/447M [00:03<00:00, 118MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1RZavefjlWwECDTHGtzT751HNg4h8m_BO
To: /content/91APP_BehaviorData_20200201.csv
100% 379M/379M [00:02<00:00, 138MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=Futur

In [ ]:
df_all = pd.DataFrame()
ymd = ['20180601', '20180701', '20180801', '20180901', '20181001', '20181101', '20181201', 
       '20190101', '20190201', '20190301', '20190401', '20190501', '20190601', '20190701', 
       '20190801', '20190901', '20191001', '20191101', '20191201', '20200101', '20200201', 
       '20200301', '20200401', '20200501', '20200601']
usecols = ['MemberId', 'HitTime', 'Behavior', 'SalePageId', 'EventTime']

#讀檔
for i in ymd:
    df = pd.read_csv('/content/91APP_BehaviorData_'+i+'.csv', usecols=usecols, low_memory=False)
    #刪除NaN
    df = df.dropna(axis=0,how='any')
    #合併df
    df_all = df_all.append([df])
    print(i+' finished')
#重新編號
df_all.index = range(len(df_all))
print(df_all)

20180601 finished
20180701 finished
20180801 finished
20180901 finished
20181001 finished
20181101 finished
20181201 finished
20190101 finished
20190201 finished
20190301 finished
20190401 finished
20190501 finished
20190601 finished
20190701 finished
20190801 finished
20190901 finished
20191001 finished
20191101 finished
20191201 finished
20200101 finished
20200201 finished
20200301 finished
20200401 finished
20200501 finished
20200601 finished
                                             MemberId        HitTime  \
0        k6hKvVelVIzj7w1Bbr/0chkKNPfcZXUr8JFHKBCJ/XE=  1530174850397   
1        k6hKvVelVIzj7w1Bbr/0chkKNPfcZXUr8JFHKBCJ/XE=  1530174850397   
2        k6hKvVelVIzj7w1Bbr/0chkKNPfcZXUr8JFHKBCJ/XE=  1530174850397   
3        rv8tOUyJAczcOgCVIkOGPuFj2cfuQCd8YLrpa5ZIO3s=  1530261414607   
4        k6hKvVelVIzj7w1Bbr/0chkKNPfcZXUr8JFHKBCJ/XE=  1530174850397   
...                                               ...            ...   
8055907  cfzPvnbUwPm0E+/08iKj/tFiKZq9bvUfZISAd

In [ ]:
#整理時間資料(注意跑很久很耗RAM)
from datetime import timedelta
df_all['HitDateTime'] = df_all['HitTime'].apply(lambda x:pd.to_datetime(x,unit='ms')+timedelta(hours=8))
df_all['EventDateTime'] = df_all['EventTime'].apply(lambda x:pd.to_datetime(x,unit='ms')+timedelta(hours=8))

KeyboardInterrupt: ignored

In [ ]:
#存df_all
try:    # 先存 excel
    fname='MembersBehavior'+".xlsx"
    writer = pd.ExcelWriter(fname)
    df_all.to_excel(writer,'Sheet1',encoding='utf-8')
    writer.save()
except:   # error 再存 pickle
    fname='MembersBehavior'+".pkl"
    with open(fname, "wb") as fp: 
        pickle.dump(df_all, fp)   
print ('save ', fname)  

In [ ]:
!gdown --id '11u1K6il5PWQ-wsObkrSzbYVNnIxgxfe5' --output MembersBehavior_timechanged.pkl

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=11u1K6il5PWQ-wsObkrSzbYVNnIxgxfe5
To: /content/MembersBehavior_timechanged.pkl
100% 405M/405M [00:01<00:00, 209MB/s]


In [ ]:
#讀整理後的pickle檔
MembersBehavior = pd.read_pickle('/content/MembersBehavior_timechanged.pkl')
MembersBehavior

,MemberId,HitTime,Behavior,SalePageId,EventTime,HitDateTime,EventDateTime
0,k6hKvVelVIzj7w1Bbr/0chkKNPfcZXUr8JFHKBCJ/XE=,1530174850397,checkout,6OPxIbYmjsX9Ec3wDKVjkA==,1530175237872,2018-06-28 16:34:10.397,2018-06-28 16:40:37.872
1,k6hKvVelVIzj7w1Bbr/0chkKNPfcZXUr8JFHKBCJ/XE=,1530174850397,checkout,6OPxIbYmjsX9Ec3wDKVjkA==,1530175071205,2018-06-28 16:34:10.397,2018-06-28 16:37:51.205
2,k6hKvVelVIzj7w1Bbr/0chkKNPfcZXUr8JFHKBCJ/XE=,1530174850397,viewproduct,6OPxIbYmjsX9Ec3wDKVjkA==,1530175313079,2018-06-28 16:34:10.397,2018-06-28 16:41:53.079
3,rv8tOUyJAczcOgCVIkOGPuFj2cfuQCd8YLrpa5ZIO3s=,1530261414607,viewproduct,SjLD9v3laYABOXQEI2C87A==,1530261682217,2018-06-29 16:36:54.607,2018-06-29 16:41:22.217
4,k6hKvVelVIzj7w1Bbr/0chkKNPfcZXUr8JFHKBCJ/XE=,1530174850397,viewproduct,KjXG2Ry1MUrRlOyBHZRurg==,1530175194380,2018-06-28 16:34:10.397,2018-06-28 16:39:54.380
...,...,...,...,...,...,...,...
8055907,cfzPvnbUwPm0E+/08iKj/tFiKZq9bvUfZISAdTEYg7M=,1593504976522,viewproduct,nqY5zlqc+/0+K5JgJ1xdRQ==,1593505322880,2020-06-30 16:16:16.522,2020-06-30 16:22:02.880
8055908,1tQkDzgRgbCO64SsNm2HYoIwlB7dU3JJ+5FYBp7heAs=,1593355890756,viewproduct,xTSO1QKeRh99Bdve52zsGw==,1593355890756,2020-06-28 22:51:30.756,2020-06-28 22:51:30.756
8055909,qZXpBeVvCXmrwcvIfO23Fk/infI8q703CLxN+kNJx/s=,1591708624924,viewproduct,agNCyhQQGVD7ZvjMXVWBxA==,1591708624924,2020-06-09 21:17:04.924,2020-06-09 21:17:04.924
8055910,oCi/x4BYShSjAyzKoAOv9JAfvFu1psLEiAsqZ39JL94=,1591101461298,viewproduct,lTU6xGMb3k+CPbU6Yum0tg==,1591101669894,2020-06-02 20:37:41.298,2020-06-02 20:41:09.894


####補資料(202007-202012)

In [ ]:
# 202007-202012
!gdown --id '1uaawxG0b5OJn9yvdho2R9PkKnga1j3tI' --output 91APP_BehaviorData_20200701.csv
!gdown --id '1Ua5GdDYfAFeUaECKcS2v0aJSgcMY1tHl' --output 91APP_BehaviorData_20200801.csv
!gdown --id '1lG3lmlxXkdFknEsoc-ZwuhB23AqOl-YY' --output 91APP_BehaviorData_20200901.csv
!gdown --id '1Ka_ha0qxumklldLBlvmD3D2c8fOcbRlG' --output 91APP_BehaviorData_20201001.csv
!gdown --id '1Tbb1GqOhBTthe7HUAODdRmGClkAsmosq' --output 91APP_BehaviorData_20201101.csv
!gdown --id '1sc4EGutXpeYHn4FhH2qIrEPf369yzCCT' --output 91APP_BehaviorData_20201201.csv

In [ ]:
df_all = pd.DataFrame()
ymd = ['20200701', '20200801', '20200901', '20201001', '20201101', '20201201']
usecols = ['MemberId', 'HitTime', 'Behavior', 'SalePageId', 'EventTime']

#讀檔
for i in ymd:
    df = pd.read_csv('/content/91APP_BehaviorData_'+i+'.csv', usecols=usecols, low_memory=False)
    #刪除NaN
    df = df.dropna(axis=0,how='any')
    #合併df
    df_all = df_all.append([df])
    print(i+' finished')
#重新編號
df_all.index = range(len(df_all))
print(df_all)

In [ ]:
#整理時間資料
from datetime import timedelta
df_all['HitDateTime'] = df_all['HitTime'].apply(lambda x:pd.to_datetime(x,unit='ms')+timedelta(hours=8))
df_all['EventDateTime'] = df_all['EventTime'].apply(lambda x:pd.to_datetime(x,unit='ms')+timedelta(hours=8))

In [ ]:
import pickle

#存df_all
try:    # 先存 excel
    fname='MembersBehavior_v2'+".xlsx"
    writer = pd.ExcelWriter(fname)
    df_all.to_excel(writer,'Sheet1',encoding='utf-8')
    writer.save()
except:   # error 再存 pickle
    fname='MembersBehavior_v2'+".pkl"
    with open(fname, "wb") as fp: 
        pickle.dump(df_all, fp)   
print ('save ', fname)

###整合201806-202012會員行為資料

In [ ]:
#補資料(202007-202012)
!gdown --id '1RSwUysCRV_CAA-Ipr7dMht6MITYTbNwZ' --output MembersBehavior_v2.pkl

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1RSwUysCRV_CAA-Ipr7dMht6MITYTbNwZ
To: /content/MembersBehavior_v2.pkl
100% 177M/177M [00:00<00:00, 210MB/s]


In [ ]:
#讀整理後的pickle檔
MembersBehavior_v2 = pd.read_pickle('/content/MembersBehavior_v2.pkl')
MembersBehavior_v2

,MemberId,HitTime,Behavior,SalePageId,EventTime,HitDateTime,EventDateTime
0,vF/x60aKXik39iSeYnGVds45jed5RZOdw1eTalj87Vg=,1594006982506,purchase,hXSHRbRq3GyJYeMIVkI06w==,1594008288006,2020-07-06 11:43:02.506,2020-07-06 12:04:48.006
1,UsT0/aQxqt6xtAGtrvFEf498JzpRz6fE/UY+9til9mw=,1596162870208,purchase,kWDCyzZQsJAB0v7LMuhA1A==,1596164184939,2020-07-31 10:34:30.208,2020-07-31 10:56:24.939
2,UsT0/aQxqt6xtAGtrvFEf498JzpRz6fE/UY+9til9mw=,1596162870208,purchase,C/W4nkuvSJThzeoIYtyPEg==,1596164184939,2020-07-31 10:34:30.208,2020-07-31 10:56:24.939
3,4jitwJ2hJKIHSQQOXPvG7bkccXsJDMIHxvMnsYfOWkg=,1595522324380,purchase,nqY5zlqc+/0+K5JgJ1xdRQ==,1595523177837,2020-07-24 00:38:44.380,2020-07-24 00:52:57.837
4,ZESBovACUfiYv1X6TzUXW1PXVXLLbCCgQz4aYNZlXnw=,1594342889816,purchase,FWIRZVZzBbIotBC5mdy75w==,1594343873195,2020-07-10 09:01:29.816,2020-07-10 09:17:53.195
...,...,...,...,...,...,...,...
3565223,QfwqMLl0w16V7PoWIFep9eB9h7pu1ALjIoCqLMpOI3M=,1607058241187,viewproduct,9xSXgYMXmi2h3ByvsMwDdg==,1607058534904,2020-12-04 13:04:01.187,2020-12-04 13:08:54.904
3565224,QfwqMLl0w16V7PoWIFep9eB9h7pu1ALjIoCqLMpOI3M=,1607058241187,viewproduct,xbasiP4uZlJhJqHXa+TeSg==,1607058316439,2020-12-04 13:04:01.187,2020-12-04 13:05:16.439
3565225,b9CRuhHm0wUIBVAQByogqnUq3stHhSmq0r7xAk9HF7I=,1608307254905,viewproduct,4cdYUFEszEeeV0YLtVRdTg==,1608308325152,2020-12-19 00:00:54.905,2020-12-19 00:18:45.152
3565226,b9CRuhHm0wUIBVAQByogqnUq3stHhSmq0r7xAk9HF7I=,1608307254905,viewproduct,sjr7/oqF9k6ZJfQKnl5Jqg==,1608308017942,2020-12-19 00:00:54.905,2020-12-19 00:13:37.942


In [ ]:
#整合201806-202012會員行為資料
MembersBehavior = MembersBehavior.append([MembersBehavior_v2])

In [ ]:
# 只留下曾有購買紀錄的member
ever_purchase_memberId_list = MembersBehavior[MembersBehavior["Behavior"]=="purchase"].MemberId.unique()
Members_df = MembersBehavior.set_index("MemberId")
Members_df = Members_df.loc[ever_purchase_memberId_list,:]
Members_df = Members_df.reset_index()

# 與關鍵字向量合併
score_dict = {"checkout":1, "viewproduct":2, "add":4, "purchase":8}
merge_df = Members_df.merge(item_df, on="SalePageId")
merge_df = merge_df.loc[:,["MemberId", "Behavior", "Vector_avg"]]
merge_df["score"] = merge_df.Behavior.apply(lambda x : score_dict[x])

merge_vector_df = merge_df.groupby("MemberId").apply(lambda x: np.average(x['Vector_avg'], weights=x['score']))
merge_vector_df

MemberId
++/PaLQ2WfBYg+I9LFaid479crfYCwavoyPpb66WVE8=    [-0.5870998865578856, 0.4482499841068473, -0.3...
++3XVjBiQPLoAXWc7gK+TdvTH4J4AmgkeFlB3mmN7YY=    [-0.5865541027303413, 0.43001356083694997, -0....
++44iA8rzzl4lYN3dzParPLqAXetHBSwDFIBUBD++t4=    [-0.6316446704114355, 0.4993911010277139, -0.4...
++6rOXDBdSgRq7Ks8WEtjF/acB6jaVRXpqWA4HRNGWw=    [-0.6268332036950264, 0.44033534201240926, -0....
++8NpYVoqyjmc8mROiPMlX/2J8JlOghaysSF3qbYCVE=    [-0.6080787635741854, 0.43907371923277666, -0....
                                                                      ...                        
zzvTsLQAKUueMvz+vx/YptKHZIMMEu7nSWnkOiBhtR4=    [-0.5095047979742762, 0.5139579494108283, -0.5...
zzw+k3PFtOtuOIu62cuXNkLNLUFDlgmXZj2XeMTwl2g=    [-0.6361055247647995, 0.5151973958891591, -0.5...
zzw9O6yOUvVbxgrvUGCnmjjacARx/bQ/12aZDz27TbE=    [-0.6310111512517067, 0.437727849397355, -0.48...
zzwvJvTcevm90Q6BkxJqud94QGdwcvDOrWT88F1cTM4=    [-0.595106887725712, 0.433846949026812, -0.470...
zzzHm2cdbc3

In [ ]:
# merge_vector_df.to_pickle("vector_df.pkl")

# Part3

In [10]:
import pandas as pd
#商品向量(Part1最終結果)
!gdown --id '1BKV0_PgPAjD729DZY3RRMZlSGMib4cID' --output item_df.pkl
#會員向量(Part2最終結果)
!gdown --id '1Kj3PEMq9LI-Vd9oTuPztyNLR9kZQC7wJ' --output vector_df.pkl
!gdown --id 1YVz3rpIKtZVxC7pxEmTUnpRPtdnGHoEn --output SalePageData.csv

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1BKV0_PgPAjD729DZY3RRMZlSGMib4cID
To: /content/item_df.pkl
100% 94.2M/94.2M [00:00<00:00, 144MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Kj3PEMq9LI-Vd9oTuPztyNLR9kZQC7wJ
To: /content/vector_df.pkl
100% 543M/543M [00:03<00:00, 154MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https

In [11]:
#讀整理後的pickle檔
members_df = pd.read_pickle('/content/vector_df.pkl')
item_df = pd.read_pickle('/content/item_df.pkl')
item_df = item_df.dropna(axis=0,how='any')

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

import os
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime, date
import random
from random import sample 
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity

raw_path = os.path.join(os.path.join('/content/gdrive/MyDrive/BDA'))

df_all = pd.DataFrame()
ymd = ['20200701', '20200801', '20200901', '20201001', '20201101', '20201201', '20210101', 
       '20210201', '20210301', '20210401', '20210501', '20210601', '20210701', '20210801', 
       '20210901', '20211001', '20211101', '20211201', '20220101', '20220201', 
       '20220301', '20220401']
usecols = ['MemberId', 'HitTime', 'Behavior', 'SalePageId', 'EventTime']

#讀檔
for i in ymd:
    df = pd.read_csv(os.path.join(raw_path, "91APP_BehaviorData_" +ｉ+ ".csv"), usecols = usecols)
    #刪除NaN
    df = df.dropna(axis=0,how='any')
    #合併df
    df_all = df_all.append([df])
    print(i+' finished')
#重新編號
df_all.index = range(len(df_all))
print(df_all)


MessageError: ignored

In [ ]:
df_all['HitTime'] = pd.to_datetime(df_all['HitTime'],unit = 'ms')
df_all["HitTime"] = df_all["HitTime"] + timedelta(hours = 8)

df_all['EventTime'] = pd.to_datetime(df_all['EventTime'],unit = 'ms')
df_all["EventTime"] = df_all["EventTime"] + timedelta(hours = 8)

In [ ]:
df_purchase = df_all[df_all["Behavior"] == "purchase"]

In [ ]:
st = datetime(2020,7,1)
et = datetime(2022,4,30)

In [ ]:
def SortByScore(sub_li):
    sub_li.sort(key = lambda x: x[1], reverse = True)
    return sub_li

In [ ]:
df_temp[df_temp['MemberId'] == ever_purchase_memberId_list[7456]]

,MemberId,HitTime,Behavior,SalePageId,EventTime,SalePageTitle
113925,U3EoJW4Sk9+vmUsM8GGWPS7g1BSBvJcNRGXv4Wos7CM=,2022-03-03 09:36:36.365,viewproduct,ITZqbB22BkCnHZnLw5J0iA==,2022-03-03 09:38:50.559,💝寵愛媽咪✨全館最高現折$399💝Ann’S質感真小羊皮優雅金鍊穆勒鞋-米白(版型偏小)
127500,U3EoJW4Sk9+vmUsM8GGWPS7g1BSBvJcNRGXv4Wos7CM=,2022-03-04 18:57:38.215,viewproduct,tbtEcM/JOHPSPu+hXRETxQ==,2022-03-04 18:59:46.461,💝寵愛媽咪✨全館最高現折$399💝Ann’S經典風格-柔軟羊皮金釦平底紳士鞋-黑(版型偏大)
127574,U3EoJW4Sk9+vmUsM8GGWPS7g1BSBvJcNRGXv4Wos7CM=,2022-03-04 18:57:38.215,viewproduct,tbtEcM/JOHPSPu+hXRETxQ==,2022-03-04 19:00:19.755,💝寵愛媽咪✨全館最高現折$399💝Ann’S經典風格-柔軟羊皮金釦平底紳士鞋-黑(版型偏大)
167277,U3EoJW4Sk9+vmUsM8GGWPS7g1BSBvJcNRGXv4Wos7CM=,2022-03-04 18:57:38.215,viewproduct,pJ/4SdI/voOx1PS0B2mmlg==,2022-03-04 18:59:55.426,💝寵愛媽咪✨全館最高現折$399💝Ann’S經典風格-柔軟羊皮金釦平底紳士鞋-米白(版型偏大)
167296,U3EoJW4Sk9+vmUsM8GGWPS7g1BSBvJcNRGXv4Wos7CM=,2022-03-08 02:11:17.879,viewproduct,pJ/4SdI/voOx1PS0B2mmlg==,2022-03-08 02:11:36.925,💝寵愛媽咪✨全館最高現折$399💝Ann’S經典風格-柔軟羊皮金釦平底紳士鞋-米白(版型偏大)
...,...,...,...,...,...,...
865512,U3EoJW4Sk9+vmUsM8GGWPS7g1BSBvJcNRGXv4Wos7CM=,2022-04-02 12:23:40.789,viewproduct,NvbTqVJ1rtMvrzNjZOeT5Q==,2022-04-02 12:23:44.698,💝寵愛媽咪✨全館最高現折$399💝Ann’S水洗牛皮-立體白色蕾絲兩穿木紋跟方頭涼鞋-白
865564,U3EoJW4Sk9+vmUsM8GGWPS7g1BSBvJcNRGXv4Wos7CM=,2022-04-02 02:46:38.163,viewproduct,NvbTqVJ1rtMvrzNjZOeT5Q==,2022-04-02 02:46:44.063,💝寵愛媽咪✨全館最高現折$399💝Ann’S水洗牛皮-立體白色蕾絲兩穿木紋跟方頭涼鞋-白
865668,U3EoJW4Sk9+vmUsM8GGWPS7g1BSBvJcNRGXv4Wos7CM=,2022-04-02 02:46:38.163,viewproduct,NvbTqVJ1rtMvrzNjZOeT5Q==,2022-04-02 02:47:10.429,💝寵愛媽咪✨全館最高現折$399💝Ann’S水洗牛皮-立體白色蕾絲兩穿木紋跟方頭涼鞋-白
865691,U3EoJW4Sk9+vmUsM8GGWPS7g1BSBvJcNRGXv4Wos7CM=,2022-04-02 12:23:40.789,viewproduct,NvbTqVJ1rtMvrzNjZOeT5Q==,2022-04-02 12:24:06.130,💝寵愛媽咪✨全館最高現折$399💝Ann’S水洗牛皮-立體白色蕾絲兩穿木紋跟方頭涼鞋-白


In [ ]:
def algo(param, deltatime):
  temp_time = st
  time_list = []
  time_list.append(st) 
  pages = pd.read_csv(os.path.join('/content', 'SalePageData.csv'))
  cnt = 0
  while True:
    if temp_time + timedelta(days = deltatime) < et:  
      temp_time = temp_time + timedelta(days = deltatime)
      time_list.append(temp_time)
    else:
      break

  for time in range(len(time_list)-1):
    df_temp = df_all[(df_all["EventTime"] > time_list[time]) & (df_all["EventTime"] < time_list[time+1])]
    df_temp = df_temp.merge(pages, left_on = "SalePageId", right_on = "SalePageId")

    df_temp_purchase = df_purchase[(df_purchase["EventTime"] > time_list[time]) & (df_purchase["EventTime"] < time_list[time+1])]
    df_temp_purchase = df_temp_purchase.merge(pages, left_on = "SalePageId", right_on = "SalePageId")

    ever_purchase_memberId_list = df_temp[df_temp["Behavior"]=="purchase"].MemberId.unique()
    random.seed(time)
    sampling_list = sample([i for i in range(len(ever_purchase_memberId_list))],1)
    
    for person in sampling_list:
      personal_temp = df_temp[df_temp["MemberId"] == ever_purchase_memberId_list[person]]
      personal_purchase_temp = df_temp_purchase[df_temp_purchase["MemberId"] == ever_purchase_memberId_list[person]]
      #print(time_list[time], time_list[time+1], personal_temp)
      # 只留下曾有購買紀錄的member
      Members_df = personal_temp.set_index("MemberId")
      Members_df = Members_df.reset_index()

      # 與關鍵字向量合併
      score_dict = {"checkout":param[0], "viewproduct":param[1], "add":param[2],"purchase":0} 
      merge_df = Members_df.merge(item_df, on="SalePageId")
      merge_df = merge_df.loc[:,["MemberId", "Behavior", "Vector_avg"]]
      merge_df["score"] = merge_df.Behavior.apply(lambda x : score_dict[x])
      merge_vector_df = merge_df.groupby("MemberId").apply(lambda x: np.average(x['Vector_avg'], weights=x['score']))
      rank_list = []
      for product in range(1, item_df.shape[0]):
          if product not in [635, 1423]:
            try:
              rank_list.append([item_df["SalePageTitle"][product],
                                float(cosine_similarity(merge_vector_df[0].reshape(1,-1), 
                                item_df["Vector_avg"][product].reshape(1,-1)))])
            except:
              pass
      sorted_rank_list = SortByScore(rank_list)
      product_preference_frame = pd.DataFrame(sorted_rank_list, columns = ["product", "similarity"]).head(10)
      
      product_list = list(product_preference_frame["product"])
      for product in product_list:
        if product in list(personal_purchase_temp["SalePageTitle"]):
          cnt += 1
  return cnt / (len(sampling_list)* (len(time_list)-1))

  #抽人出來算該會員的向量（去search item_df，並加權平均）
  #對於每個人output我們推的產品
  #對應該會員購買的資料，去計算準確率以及複雜版本的指標


In [ ]:
max_val = 0
for i in range(1,10):
  for j in range(i,10):
    for k in range(j,10):
      param = [i, j, k]
      cur_val = algo(param,60)
      if cur_val > max_val:
        print(cur_val, i,j,k)
        max_val = cur_val

0.6363636363636364 1 1 1
